In [ ]:
import codecs, json, csv
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pandas as pd, csv
import string as string

import re

from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize


from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


In [ ]:
twitterscraper "(Presiden OR Menteri OR Pemerintahan OR Jokowi) AND (Tolong OR Mohon)" -l 100 -p 400 -bd 2017-01-01 -ed 2017-06-01 -o dengantolong2.json

In [51]:
with codecs.open('dengantolong2.json', 'r', 'utf-8') as f:
    tweets = json.load(f, encoding='utf-8')

list_tweets = [list(elem.values()) for elem in tweets]
list_columns = list(tweets[0].keys())
df = pd.DataFrame(list_tweets, columns=list_columns)
df.to_csv('dengantolong.csv')
tweets[1]

list_tweet = []
for i in range(len(tweets)):
    list_tweet.append([tweets[i]['user'],'1', tweets[i]['text']])
    
df = pd.DataFrame(list_tweet, columns=['name','class','text'])
# df.to_csv('dengantolong.csv')

NameError: name 'codecs' is not defined

In [52]:
# start here

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import numpy as np
import pickle
import random


In [58]:
with open('Indonesian_Manually_Tagged_Corpus.tsv', 'r', encoding='utf-8') as tsvfile :
    rows = tsvfile.read().split('\n')
    
pair = []
sentences = []

for each in rows:
    if each == '':
        sentences.append(pair)
        pair = [] 
    else:
        word, tag = each.split('\t')
        pair.append([word, tag])

In [59]:
# Extract Feature

In [62]:
def neighbor_word(sentence, i):
    if i == 0 :
        prev_word = ''
    else :
        prev_word = sentence[i-1][0]
    if i == len(sentence)-1 :
        next_word = ''
    else :
        next_word = sentence[i+1][0]
    return {'prev_word' : prev_word, 'next_word' : next_word}

def morphems(word):
    prefix_1 = word[0]
    prefix_2 = word[:2]
    prefix_3 = word[:3]
    prefix_4 = word[:4]
    suffix_1 = word[-1]
    suffix_2 = word[-2:]
    suffix_3 = word[-3:]

    return {'prefix_1' : prefix_1, 'prefix_2' : prefix_2, 'prefix_3' : prefix_3, 'prefix_4' : prefix_4, 'suffix_1' : suffix_1, 'suffix_2' : suffix_2, 'suffix_3' : suffix_3}

def has_hyphen(word):
    return {'has_hyphen' : '-' in word}

def is_numeric(word):
    return {'is_numeric' : word.isdigit()}

def word_case(word):
    is_uppercase = word.upper() == word
    is_lowercase = word.lower() == word
    is_capitalized = word[0].upper() == word[0]
    return {'is_uppercase' : is_uppercase,'is_lowercase' : is_lowercase, 
            'is_capitalized' : is_capitalized}


def word_position(sentence, index):
    if (index == 0):
        pos = 0
        prev_pos = -1
        next_pos = 1
    elif (index == len(sentence)):
        pos = 2
        prev_pos = 1
        next_pos = -1
    else :
        if(index == 1):
            prev_pos = 0
            pos = 1
            next_pos = 2
        elif(index == len(sentence)-1):
            prev_pos = 1
            pos = 2
            next_pos = -1
        else :
            prev_pos = pos = next_pos = 1
    return {'prev_pos' : prev_pos, 'pos' : pos, 'next_pos' : next_pos}

def feature_extractor(sentences):
    X = []
    y = []
    for sentence in sentences:
        for i in range(len(sentence)):
            features = {}

            word = sentence[i][0]
            features.update({'value': word})
            
            features.update(neighbor_word(sentence, i))
            features.update(word_position(sentence, i))
            features.update(morphems(word))
            features.update(word_case(word))
            features.update(has_hyphen(word))
            features.update(is_numeric(word))
            
            X.append(features)
            y.append(sentence[i][1])
    return X, y
X, y = feature_extractor(sentences)

In [63]:
X[1]

{'value': 'untuk',
 'prev_word': 'Kera',
 'next_word': 'amankan',
 'prev_pos': 0,
 'pos': 1,
 'next_pos': 2,
 'prefix_1': 'u',
 'prefix_2': 'un',
 'prefix_3': 'unt',
 'prefix_4': 'untu',
 'suffix_1': 'k',
 'suffix_2': 'uk',
 'suffix_3': 'tuk',
 'is_uppercase': False,
 'is_lowercase': True,
 'is_capitalized': False,
 'has_hyphen': False,
 'is_numeric': False}

In [64]:
# Vectorize Feature

In [65]:
dict_vect = DictVectorizer(sparse=False)
size = 5000
y = y[0:size]
X = dict_vect.fit_transform(X[0:size])

print(len(X), len(y))

5000 5000


In [66]:
from sklearn.model_selection import train_test_split

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [68]:
model = LogisticRegression()
model.fit(X_train, y_train)

model_pred = model.predict(X_test)
pickle.dump(model, open("post_tag_model.sav", "wb"))
print('Accuracy Score Logistic : ', accuracy_score(model_pred, y_test))

Accuracy Score Logistic :  0.9064
